In [15]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# set executable path to NASA Mars News
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - Driver [C:\Users\Edgar\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [3]:
# letss visit the url
url = 'https://redplanetscience.com'

# instruct bs4 to visit url
browser.visit(url)

# use an optional clause to delay loading the page
# first look for elements with combination of <div> and <list_text>-----e.g. (ul.item_list) found as (<ul class='item_list')
# also tells  browser to wait for 1 sec before searching components

browser.is_element_present_by_css('div.list_text', wait_time=1) 

True

In [4]:
# next we will tell bs4 to parse/analyse (using html.parser) text and store in an object (news_soup)

html = browser.html

news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')   # variable looks for <div> and its descedants (other elements within <div>)
                                                     # div.select one points to <div> with class 'list_text' li
    

In [5]:
# we have our text/code in slide_elem---we will look for <div> element and class=content_title in this variable

slide_elem.find('div', class_='content_title')   # output is an html containing content title inside <div. element

<div class="content_title">How NASA's Mars Helicopter Will Reach the Red Planet's Surface</div>

In [6]:
# the title is somewhere in the html code... thats all we need not the html stuff
# first we will find <a> tag and save it in an object ---news_title

news_title = slide_elem.find('div', class_='content_title').get_text()    # .get_text only returns the text

news_title


"How NASA's Mars Helicopter Will Reach the Red Planet's Surface"

In [7]:
# now lets look for the summary (teser) using find
# when we find 'teser' from DevTools we get the first summary/teser----article_teaser_body

news_p = slide_elem.find('div', class_='article_teaser_body').get_text()

news_p

'The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.'

# Featured Images
### after scraping text, lets now add images

In [8]:
# when we open the NASA page, the first image we see is the Featured Image----we want the fullblown image (use its link for url)
# we will first visit the webpage/url---a new page opens with full blown image

url = 'https://spaceimages-mars.com'
browser.visit(url)

## Using buttons

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

https://spaceimages-mars.com/image/featured/mars3.jpg

In [12]:
# parse resulting html with soup
html = browser.html

img_soup = soup(html, 'html.parser')

In [13]:
# we now need the relative image url
img_url_rel = img_soup.find('img',class_='fancybox-image').get('src')

img_url_rel

'image/featured/mars3.jpg'

In [14]:
# use base URL to create absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

## Getting Mars Facts
### We will be scrapping table content
### A table is made up of smaller containers (tbody) nested in the table tag
###  tr is the tag for each table---table data is stored in td tags----these esablish the columns

In [16]:
# instead of scraping each row, we read the html <td> tags using pythons panda

df = pd.read_html('https://galaxyfacts-mars.com')[0]  # this is the mars facts webpage (0 - tells panda to pull first table)

df.columns=['description', 'Mars', 'Earth']

df.set_index('description', inplace=True)

df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [18]:
# we will need to add this clean table into a webpage--any changes on the table will be reflected in the webpage
# we will have to turn it back to html using .to_hmtl
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

## Shut down the  browser
#### Browser should always be shut else it will continue using computer resources

In [19]:
# to shut down the browser
browser.quit()

## Automating The Scraping process
#### Jupyter is an awesome tool but we cannot automate scraping using jupyter
#### we have to convert this file into Pythons .py file
##### On this page go to File--->Download as---->Python (.py)